<a href="https://colab.research.google.com/github/wanlichen2024/gdda612_project1/blob/main/612_assessment_1_webscraping%201%20ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin
import lxml.html
import csv
import json

In [ ]:
url = 'https://www.noelleeming.co.nz/c/top-brands/apple'
request = requests.get(url)
request.status_code

200

In [ ]:
'''
https://www.noelleeming.co.nz/c/top-brands/apple
https://www.noelleeming.co.nz/c/top-brands/apple?start=32
https://www.noelleeming.co.nz/c/top-brands/apple?start=64
'''

'\nhttps://www.noelleeming.co.nz/c/top-brands/apple\nhttps://www.noelleeming.co.nz/c/top-brands/apple?start=32\nhttps://www.noelleeming.co.nz/c/top-brands/apple?start=64\n'

In [ ]:
url = 'https://www.noelleeming.co.nz/c/top-brands/apple'
items_per_page = 32  # number of product at each page

# initiate a list to put all the useful information
all_items = []

# total 18 pages
total_pages = 18

# a loop go through 18 page
for page_number in range(total_pages):
    if page_number == 0:
        url1 = url
    else:
        start_value = page_number * items_per_page
        url1 = f'{url}?start={start_value}'

    # request every page content
    response = requests.get(url1)
    if response.status_code != 200:
        print(f"Failed to retrieve page {page_number}")
        continue

    # using beautifil soup to extract every page
    soup = BeautifulSoup(response.content, 'html.parser')

    # find all product tile
    product_tiles = soup.find_all('div', class_='product-tile')

    # extract each product information
    for tile in product_tiles:
        # extract product name
        name_tag = tile.find('a', class_="link text-emphasized")
        product_name = name_tag.get_text(strip=True) if name_tag else 'N/A'

        # extract price
        try:
            product_data = tile['data-gtm-product']
            product_info = json.loads(product_data)
            product_price = product_info.get('price', 'N/A').replace('$', '').replace(',', '')
        except (KeyError, json.JSONDecodeError):
            product_price = 'N/A'

        # extract rating
        rating_tag = tile.find('span', class_="sr-only selected-assistive-text")
        product_rating = rating_tag.get_text(strip=True).split(':')[-1].strip() if rating_tag else 'N/A'

        # input inforamation to the list
        all_items.append({
            'product_name': product_name,
            'product_price': product_price,
            'product_rating': product_rating
        })

    print(f"Page {page_num} processed successfully.")

# save as csv format
with open('all_productsl_lm.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=['product_name', 'product_price', 'product_rating'])
    writer.writeheader()
    writer.writerows(all_items)

print("All information has been saved all_products_lm.csv")



Page 0 processed successfully.
Page 1 processed successfully.
Page 2 processed successfully.
Page 3 processed successfully.
Page 4 processed successfully.
Page 5 processed successfully.
Page 6 processed successfully.
Page 7 processed successfully.
Page 8 processed successfully.
Page 9 processed successfully.
Page 10 processed successfully.
Page 11 processed successfully.
Page 12 processed successfully.
Page 13 processed successfully.
Page 14 processed successfully.
Page 15 processed successfully.
Page 16 processed successfully.
Page 17 processed successfully.
All information has been saved all_products_lm.csv


In [ ]:
df = pd.read_csv('all_productsl_lm.csv')

In [ ]:
df.shape

(576, 3)

In [ ]:
df = df.to_csv('/content/drive/MyDrive/all_products_lm.csv', index=False)